In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import split
import pandas as pd
import matplotlib.pyplot as plt
import upsetplot

# Create a local SparkSession (seulement le master et pas les slaves)
# spark = SparkSession.builder.master("local").appName("test").getOrCreate() 

#créer une session dans le master
spark = SparkSession.builder \
    .master("spark://172.20.53.96:7077") \
    .appName("WDC-intersection") \
    .config("spark.executor.memory","28g") \
    .config("spark.driver.memory","28g") \
    .getOrCreate()
#spark = SparkSession.builder.master("local").appName("WDC-intersection").getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions",1000)

#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = '***REMOVED***'
aws_secret_access_key = '***REMOVED***'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')
import tldextract

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/30 18:54:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/30 18:54:43 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
from pyspark.sql.functions import col, count
from pyspark.sql import functions as f

from pyspark.sql import Row
import re
from urllib.parse import urlparse



quad_motif = re.compile(r'([^\s]+)\s([^\s]+)\s(.+)\s([^\s]+)\s+\.')

def parseQ(l):
    result=quad_motif.match(l)
    if result : 
        parsed_result = result.group(4).strip().strip("<>")
        #output = urlparse(parsed_result, 'HOST').netloc
        tld = tldextract.extract(parsed_result)
        if tld.registered_domain:
            output = tld.registered_domain
        else:
            # in case the URL is an IP
            output = tld.domain
        return Row(domain = output)
    else:
        return None
# parse_url(, 'HOST')




In [3]:

test_data = [
    "<http://kevin.com> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> 'Some data' <http://10.0.0.1:4040/test?data=value>  .",
    "<http://kevin.com> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> 'Some data' <https://fr.prime.amazon.co.uk/test?data=value>  .",
    "<http://kevin.com> <http://schema.org/knows> 13.5 <172.16.0.1/test?data=value>  ."
]
test_rdd = spark.sparkContext.parallelize(test_data)
test = test_rdd.map(lambda l: parseQ(l)).toDF()
test.show()

+------------+
|      domain|
+------------+
|    10.0.0.1|
|amazon.co.uk|
|  172.16.0.1|
+------------+



In [4]:
# File file:/webdatacommons_data/home/nathan/petitGrapheRDFavant.txt does not exist
#spark.sparkContext.addFile("petitGrapheRDFavant.txt")

#linesavant = spark.sparkContext.textFile(f"petitGrapheRDFavant.txt")
linesavant = spark.sparkContext.textFile(f"s3a://wdc/**")

spavant=linesavant.map(lambda l: parseQ(l)).filter(lambda x: x is not None).toDF()
spavant.createOrReplaceTempView("Superavant")
spavant.show()

#linesapres = spark.sparkContext.textFile(f"petitGrapheRDFapres.txt")
linesapres = spark.sparkContext.textFile(f"s3a://wdc-2023/**")

spapres=linesapres.map(lambda l: parseQ(l)).filter(lambda x: x is not None).toDF()
spapres.createOrReplaceTempView("Superapres")
spapres.show()

23/11/30 18:54:50 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+-----------------+
|           domain|
+-----------------+
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
|1000lifehacks.com|
+-----------------+
only showing top 20 rows



+----------------+
|          domain|
+----------------+
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|          sch.gr|
|888sportkor.info|
+----------------+
only showing top 20 rows



In [5]:
from pyspark.sql.functions import udf



'''
@udf('string')
def extract_tld(url):
    return tldextract.extract(url).registered_domain

spark.udf.register("extract_tld", extract_tld)
'''
result = spark.sql("""
    SELECT (
        COUNT(DISTINCT CASE WHEN a.parsed_domain IS NOT NULL AND b.parsed_domain IS NULL THEN a.parsed_domain END) * 100 / COUNT(DISTINCT a.parsed_domain)
    ) AS percentage_after_not_before,
    (
        COUNT(DISTINCT CASE WHEN b.parsed_domain IS NOT NULL AND a.parsed_domain IS NULL THEN b.parsed_domain END) * 100 / COUNT(DISTINCT b.parsed_domain)
    ) AS percentage_before_not_after
    FROM (
        SELECT DISTINCT domain AS parsed_domain FROM Superapres
    ) a
    FULL OUTER JOIN (
        SELECT DISTINCT domain AS parsed_domain FROM Superavant
    ) b
    ON a.parsed_domain = b.parsed_domain
""")

result.show()

+---------------------------+---------------------------+
|percentage_before_not_after|percentage_after_not_before|
+---------------------------+---------------------------+
|          36.92331302220138|           34.8743341275519|
+---------------------------+---------------------------+

